# Systematic Search

In the last session, we've prepared different classes that are useful to solve search problems. You've completed the `Node` class that we can use now for our first search strategy **breadth_first_graph_search**.

Compare your version with the one found on my [Github](https://github.com/iaherzog/search/blob/main/search.py) repository. 

For the following exercises, we will clone the github repo and use the node class from the solution.






In [65]:
#!git clone https://github.com/iaherzog/search.git
import sys
sys.path.append('content/search')
!dir

 Volume in drive C is Windows
 Volume Serial Number is F8CF-6D3D

 Directory of c:\Users\denis\source\repos\AISO-HSLU\Search Algorithms

12/05/2024  14:02    <DIR>          .
12/05/2024  13:32    <DIR>          ..
12/05/2024  14:03            18'911 01_systematic_search.ipynb
12/05/2024  14:02    <DIR>          content
               1 File(s)         18'911 bytes
               3 Dir(s)  94'061'838'336 bytes free


## Breadth-first search

First, you are going to implement the breadth-first search strategy.

Hints: 


- Create the queue for the frontier using `collections.py`. It implements high performance data types. The `collection.deque` allows you to easily extend the queue with `frontier.append` and to remove items from the queue with `frontier.popleft()`. An example of how to use this class is given below.
- The `breath_first_graph_search` function takes a `problem` as an argument and returns the goal node if it is found. The template for the function is given below.
- Remember that you can access the children of a node with the following code: `node.expand(problem)`.

In [26]:
from collections import deque

my_queue = deque()
my_queue.append('first_item')
my_queue.append('second_item')
my_queue.append('third_item')

print('get and remove first item')
print(my_queue.popleft())

print('get and remove first item now')
print(my_queue.popleft())

get and remove first item
first_item
get and remove first item now
second_item


In [48]:
class Graph:

    """A graph connects nodes by edges.  Each edge can also
    have a length associated with it.  The constructor call is something like:
        g = Graph({'A': {'B': 1, 'C': 2})
    this makes a graph with 3 nodes, A, B, and C, with an edge of length 1 from
    A to B,  and an edge of length 2 from A to C.  You can also do:
        g = Graph({'A': {'B': 1, 'C': 2}, directed=False)
    This makes an undirected graph, so inverse links are also added. The graph
    stays undirected; if you add more edges with g.connect('B', 'C', 3), then
    inverse link is also added.  You can use g.get_nodes() to get a list of nodes,
    g.get_edges('A') to get a dict of edges out of A, and g.get_distance('A', 'B') to get the
    length of the edge from A to B. """

    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    def make_undirected(self):
        """Make a digraph into an undirected graph by adding symmetric edges."""
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.add_connection(b, a, dist)

    def connect(self, A, B, distance=1):
        """Add a link from A and B of given distance, and also add the inverse
        link if the graph is undirected."""
        self.add_connection(A, B, distance)
        if not self.directed:
            self.add_connection(B, A, distance)

    def add_connection(self, A, B, distance):
        """Add a link from A to B of given distance, in one direction only."""
        self.graph_dict.setdefault(A, {})[B] = distance

    def get_edges(self, a):
        return self.graph_dict.setdefault(a, {})

    def get_distance(self, a, b):
        links = self.graph_dict.setdefault(a, {})
        return links.get(b)

    def get_nodes(self):
        """Return a list of nodes in the graph."""
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)


def UndirectedGraph(graph_dict=None):
    """Build a Graph where every edge (including future ones) goes both ways."""
    return Graph(graph_dict = graph_dict, directed=False)


class GraphProblem():

    """The problem of searching a graph from one node to another."""

    def __init__(self, initial, goal, graph):
        self.initial = initial
        self.goal = goal
        self.graph = graph

    def goal_test(self, state):
        """Return True if the state is the goal. """
        return state == self.goal

    def get_actions_from(self, A):
        """The actions at a graph node are just its neighbors."""
        return list(self.graph.get_edges(A).keys())

    def get_path_cost(self, A, B):
        return self.graph.get_distance(A, B)



class Node:
    """A node in a search tree."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        actions = problem.get_actions_from(self.state)
        successors = []
        for action in actions:
            successors.append(self.create_child_node(problem, action))
        return successors

    def create_child_node(self, problem, action):
        next_state = action
        next_node = Node(next_state, parent=self, action=action,
                         path_cost=self.path_cost + problem.get_path_cost(self.state, next_state))
        return next_node

    def get_path_from_root(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    def get_solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.get_path_from_root()[1:]]


Use the template below to implement the breadth first search algorithm.

In [91]:
from collections import deque

def breadth_first_graph_search(problem):
    frontier = deque([Node(problem.initial)])
    explored = set()
    if problem.goal_test(problem.initial):
        return frontier[0]
    while frontier:
        node = frontier.popleft()
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    return child
                frontier.append(child)
    print("Explored length:" + len(explored))
    return None


Lets create a map from the text book example to test if the algorithm is working.

In [49]:
romania_graph = UndirectedGraph(dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)))

With this, we can test our search algorithm. We define our search problem with the initial state Sibiu, the goal state Bucharest and the undirected graph romania_graph. Let's find the solution with the breadth-first search.

In [51]:
start = 'Sibiu'
goal = 'Bucharest'
problem = GraphProblem(start, goal, romania_graph)
goal_node = breadth_first_graph_search(problem)

The following code will show you some information about the solution and help you to troubleshoot your algorithm:

In [52]:
def evaluate(node):
    if node:
        print("The search algorithm reached " + node.state + " with a cost of " + str(node.path_cost) + ".")
        print("The actions that led to the solutions are the following: ")
        print(node.get_solution())
    else: 
        print('no solution found')
        
evaluate(goal_node)

The search algorithm reached Bucharest with a cost of 310.
The actions that led to the solutions are the following: 
['Fagaras', 'Bucharest']


Compare the solution with the lecture slides (but note: here, Sibiu has more connections). If your solution is correct, you have successfully implemented your first search algorithm!

To evaluate the performance of your search algorithm, add the following features:

- print the depth of the solution found
- count how many nodes were visited
- print the maximum number of nodes that were stored at the same time



## Swiss Railway System ##

Let's try the algorithm on a larger data set. I've created a SBB class that can be used to import the data from the json file provided by the open data initiative of the swiss federal railways:



In [82]:
import json
import operator
import math


class TrainLine:
    def __init__(self, id, name):
        self.id = id
        self.name = name
        self.hubs = []
        self.hub_location = dict()

    def add_hub_at_km(self, hub, km):
        self.hubs.append(hub)
        self.hub_location[hub.name] = km

    def get_sorted_hubs(self):
        return sorted(self.hub_location.items(), key=operator.itemgetter(1))


class Hub:

    def __init__(self, name="", x=0, y=0):
        self.name = name
        self.x = x
        self.y = y

    def get_coordinates(self):
        return self.x, self.y


class SBB:
    def __init__(self):
        self.hubs = dict()
        self._train_lines = dict()

    def import_data(self, json_file_name):
        with open(json_file_name) as f:
            lines = json.load(f)
        for j in lines:
            if 'fields' not in j:
                continue
            train_line_id = j['fields']['linie']
            if train_line_id not in self._train_lines:
                train_line_name = j['fields']['linienname']
                self._train_lines[train_line_id] = TrainLine(train_line_id, train_line_name)
            hub = Hub()
            hub.name = treat_string(j['fields']['bezeichnung_bpk'])
            hub.x = j['fields']['geopos'][0]
            hub.y = j['fields']['geopos'][1]
            km = j['fields']['km']

            self._train_lines[train_line_id].add_hub_at_km(hub, km)
            self.hubs[hub.name] = hub

        print('successfully imported ' + str(len(self.hubs)) + ' hubs')
        print('successfully imported ' + str(len(self._train_lines)) + ' train lines')

    def create_map(self):
        map = dict()
        for line in self._train_lines:
            previous_hub_name = ""
            previous_km = -1
            for h in self._train_lines[line].get_sorted_hubs():
                hub_name = h[0]
                km = h[1]
                if previous_hub_name:
                    distance = abs(km - previous_km)
                    map.setdefault(hub_name, dict())
                    map.setdefault(previous_hub_name, dict())
                    map[hub_name].setdefault(previous_hub_name)
                    map[previous_hub_name].setdefault(hub_name)
                    map[hub_name][previous_hub_name] = distance
                    map[previous_hub_name][hub_name] = distance
                previous_hub_name = hub_name
                previous_km = km
        return map

    def get_hub_locations(self):
        locations = dict()
        for h in self.hubs:
            locations[h] = self.hubs[h].get_coordinates()
        return locations

    def get_distance_between(self, h1, h2):
        return math.sqrt((self.hubs[h1].x - self.hubs[h2].x) ** 2 + (self.hubs[h1].y - self.hubs[h2].y) ** 2) * 100


def treat_string(name):
    name = name.replace(" ", "_")
    name = name.replace('(', "")
    return name.replace(')', "")


In [62]:
import os
os.getcwd()

'c:\\Users\\denis\\source\\repos\\AISO-HSLU\\Search Algorithms'

In [84]:
sbb = SBB()
sbb.import_data('content/search/linie-mit-betriebspunkten.json')

successfully imported 2787 hubs
successfully imported 401 train lines


The object `sbb` contains all the hubs and trainlines. For each hub, the x- and y-coordinates are given. To visualize the hubs, we can use the [folium](https://python-visualization.github.io/folium/modules.html) library.

In [69]:
%pip install folium

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/100.0 kB ? eta -:--:--
   ------------ --------------------------- 30.7/100.0 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 100.0/100.0 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/82.0 kB ? eta -:--:--
   ---------------------------------------- 82.0/82.0 kB ? eta 0:00:00


In [85]:
import folium
map_ch = folium.Map(location=[46.8, 8.33],
                    zoom_start=8, tiles="Stamen Toner")

for hub in sbb.hubs:
    folium.CircleMarker(location=[sbb.hubs[hub].x, sbb.hubs[hub].y],
                        radius=2,
                        weight=4).add_to(map_ch)
map_ch


ValueError: Custom tiles must have an attribution.

In this exercise, we are not restricted to the official train lines. If two hubs are connected, we can go from one hub to the other. If you have successfully implemented the classes above, the following code should execute and provide the directions between Rotkreuz and Thalwil.

In [113]:
from collections import deque

def breadth_first_graph_search1(problem):
    frontier = deque([Node(problem.initial)])
    explored = set()
    if problem.goal_test(problem.initial):
        return frontier[0]
    while frontier:
        node = frontier.popleft()
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    print("frontier:" + str(len(frontier)))
                    print("Explored nodes:" + str(len(explored)))
                    return child
                frontier.append(child)
    return None

start = 'Rotkreuz'
goal = 'Thalwil'
sbb_graph = UndirectedGraph(sbb.create_map())
problem = GraphProblem(start, goal, sbb_graph)
goal_node = breadth_first_graph_search1(problem)

frontier:19
Explored nodes:136


Let's print and visualize the solution.

In [93]:
evaluate(goal_node)

def show_solution(map, goal_node):
      
    points = []
    
    for hub in goal_node.get_path_from_root():
        points.append([sbb.hubs[hub.state].x, sbb.hubs[hub.state].y])
        folium.CircleMarker(location=[sbb.hubs[hub.state].x, sbb.hubs[hub.state].y], color='red',
                        radius=2,
                        weight=4).add_to(map)
    folium.PolyLine(points, color='red').add_to(map)
    return map

m = show_solution(map_ch, goal_node)
m

The search algorithm reached Thalwil with a cost of 36.906.
The actions that led to the solutions are the following: 
['Hunenberg_Chamleten', 'Hunenberg_Zythus', 'Cham', 'Cham_Alpenblick', 'Zug_Chollermuli', 'Zug_Schutzengel', 'Zug', 'Zug_Nord_Abzw', 'Baar_Lindenpark', 'Baar_Neufeld', 'Baar', 'Litti_Baar', 'Sihlbrugg', 'Horgen_Oberdorf', 'Oberrieden_Dorf', 'Thalwil']


AttributeError: 'list' object has no attribute 'add_child'

DFS Search Algorithm

In [74]:
def depth_first_graph_search(problem):
  frontier = [Node(problem.initial)]
  explored = set()
  while frontier:
      node = frontier.pop()
      if problem.goal_test(node.state):
          return node
      explored.add(node.state)
      successors = node.expand(problem)
      for successor in successors:
          if successor.state not in explored:
              frontier.append(successor)
  return None

In [101]:
start = 'Rotkreuz'
goal = 'Thalwil'
sbb_graph = UndirectedGraph(sbb.create_map())
problem = GraphProblem(start, goal, sbb_graph)
goal_node = depth_first_graph_search(problem)

NameError: name 'depth_first_graph_search1' is not defined

In [96]:
evaluate(goal_node)

The search algorithm reached Thalwil with a cost of 1362.7200000000005.
The actions that led to the solutions are the following: 
['Bruglen_Spw', 'Immensee_West_Abzw', 'Kussnacht_am_Rigi', 'Merlischachen', 'Meggen', 'Meggen_Zentrum', 'Luzern_Verkehrshaus', 'Gutsch_Abzw', 'Fluhmuhle_Abzw', 'Emmenbrucke', 'Emmenbrucke_Gersag', 'Hubeli_LU', 'Waldibrucke', 'Eschenbach', 'Ballwil', 'Hochdorf_Schonau', 'Hochdorf', 'Baldegg_Kloster', 'Baldegg', 'Gelfingen', 'Hitzkirch', 'Ermensee', 'Mosen', 'Beinwil_am_See', 'Birrwil', 'Boniswil', 'Boniswil_Nord', 'Hallwil', 'Seon', 'Lenzburg_Seetal', 'Lenzburg_West_Abzw', 'Hunzenschwil', 'Suhr', 'Oberentfelden', 'Kolliken', 'Kolliken_Oberdorf', 'Safenwil', 'Walterswil-Striegel', 'Kungoldingen', 'Zofingen_Nord_Abzw', 'Aarburg-Oftringen_Sud_Abzw', 'Aarburg-Oftringen', 'Aarburg-Oftringen_West_Abzw', 'Bifang_AG_Spw', 'Rothrist', 'Rothrist_West_Abzw', 'Wanzwil_Abzw', 'Aespli', 'Oberhard_BE', 'Mattstetten', 'Schonbuhl_SBB', 'Zollikofen_Nord_Abzw', 'Zollikofen', 'R

##  More Uninformed Search Algorithms ##

As you know, the breadth-first search algorithm is just one of several systematic search strategies. Implement the following search algorithms and evaluate their performance. You might have to use the depth of the search tree.

1. Breadth-First Search (BFS) - already done :D
1. Depth-First Search (DFS)
1. Depth-Limited Search (DLS)
1. Iterative Deepening Search (IDS)


**TESTAT**: For the testat exercice on ILIAS, you only need to implement the first two algorithms.

Additional Questions: 
- What is special about the sbb railway map in terms of complexity (branching factor, depth)?
- How could you preprocess the data set in order to reduce the search space?
